In [1]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import posexplode
from pyspark.sql.functions import spark_partition_id
from pyspark.sql.functions import substring
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark=SparkSession.builder.getOrCreate()

store_df=spark.read.format("csv").option("header", True).load("stores.csv")

sales_df=spark.read.format("csv").option("header", True).load("sales.csv")

store_df=store_df.withColumn("salt_value", concat("store_id", lit("_"), (rand()*0).cast("Integer")))
#store_df.show()

sales_df=sales_df.withColumn("sequence", array([lit(i) for i in range(0,3)]))
sales_df=sales_df.withColumn("exploded_values", explode("sequence"))

sales_df=sales_df.withColumn("salt_value2", concat("store_id", lit("_"), "exploded_values"))
#sales_df.show()

join_df=store_df.join(sales_df, store_df.salt_value==sales_df.salt_value2, "inner")
join_df.show()

+--------+----------+----------+--------+----------+------+---------+---------------+-----------+
|store_id|store_name|salt_value|store_id|product_id|amount| sequence|exploded_values|salt_value2|
+--------+----------+----------+--------+----------+------+---------+---------------+-----------+
|     101|   Walmart|     101_0|     101|      P001|   100|[0, 1, 2]|              0|      101_0|
|     101|   Walmart|     101_0|     101|      P002|   200|[0, 1, 2]|              0|      101_0|
|     101|   Walmart|     101_0|     101|      P003|   150|[0, 1, 2]|              0|      101_0|
|     102|    Target|     102_0|     102|      P004|   300|[0, 1, 2]|              0|      102_0|
|     103|    Costco|     103_0|     103|      P005|   400|[0, 1, 2]|              0|      103_0|
|     101|   Walmart|     101_0|     101|      P006|   500|[0, 1, 2]|              0|      101_0|
|     104|   BestBuy|     104_0|     104|      P007|   250|[0, 1, 2]|              0|      104_0|
+--------+----------